### Convert Python to Rust Using LLMs

In this blog post, we will walk through how to use Large Language Models (LLMs) to translate Python code into Rust code.

**Prerequisites**

1. Install langchain and gradio requirements:

   ````!pip install langchain-openai langchain gradio````
   
2. **Make sure Rust is installed:**

   You can install Rust from the [official website](https://www.rust-lang.org/), or alternatively, you can use a Rust REPL (though this is not optimal). You can use a REPL for Rust but it is not optimal [check this reddit](https://www.reddit.com/r/rust/comments/17lo0gg/evcxr_rust_repl_is_an_incredible_tool/).


In [133]:
import subprocess
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage,AIMessage,SystemMessage
import gradio as gr 
import time
import os
import sys
import io

In [27]:
load_dotenv()
os.environ["OPENAI_API_KEY"]=os.getenv("OPENAI_API_KEY")

In [41]:
llm=ChatOpenAI(temperature=0,streaming=True)

In [128]:
SYSTEM_PROMPT="""You are a helpful assistant. Your task is to convert Python code into equivalent Rust code. 
The code you generate should produce the same output as the given Python code. 
Respond only with Rust code; do not explain your work other than a few comments."""
    

In [129]:
USER_PROMPT="""Convert the following code to equivalent Rust: {py_code}"""

### Define functions

In [58]:
py_code="""
import time
def alternating_harmonic_sum(n):
    "Compute the sum of the alternating harmonic series up to n (inefficient for large n)."
    total = 0
    for i in range(1, n + 1):
        total += (-1) ** (i + 1) / i
    return total

# Parametres
n = 10_000_000  # 
start = time.time()
result = slow_alternating_harmonic_sum(n)
end = time.time()

print(f"Alternating harmonic sum up to {n}: {result}")
print(f"Time taken: {end - start:.2f} seconds")
"""

In [94]:
def call_llm(py_code: str)->None:
    messages=[SystemMessage(content=system_prompt)]+[HumanMessage(content=user_prompt.format(py_code=py_code))]
    output=""
    for chunk in llm.stream(messages):
        output+=chunk.content
        yield output

In [97]:
def write_file(code:str)->None:
    code=code.replace("```rust","").replace("```","")
    with open("rust.rs","w") as f:
        f.write(code)

In [132]:
def run_rust(rust_code:str)->str:
    rust_file_path="rust.rs"
    write_file(rust_code)
    
    compile_process = subprocess.run(["rustc", rust_file_path], capture_output=True, text=True)
    
    # Check for compilation errors
    if compile_process.returncode != 0:
        print("Compilation failed:")
        return(compile_process.stderr)
    else:
        #  Run the compiled binary
        compiled="./"+rust_file_path.split(".")[0]+".exe"
        output = subprocess.run([compiled], capture_output=True, text=True)
        
        os.remove(compiled)
        os.remove(rust_file_path)
        return(output.stdout)

In [134]:
def run_python(code:str)->None:
    try:
        output = io.StringIO()
        sys.stdout = output
        exec(code)
    finally:
        sys.stdout = sys.__stdout__
    return output.getvalue()

### Define a simple UI with Gradio

In [126]:
with gr.Blocks() as ui:
    with gr.Row():
        title = gr.HTML("<h1 style='text-align: center;'>Python to Rust</h1>")
    with gr.Row():
        python_code=gr.Textbox(label="Python code",lines=8)
        rust_code=gr.Textbox(label="Rust code",lines=8)
    with gr.Row():
        submit=gr.Button("Convert code")
    with gr.Row():
        python_result=gr.Textbox(label="Python code",lines=4)
        rust_result=gr.Textbox(label="Rust Result",lines=4)
    with gr.Row():
        run_python_button=gr.Button("Run Python")
        run_rust_button=gr.Button("Run Rust")
    submit.click(call_llm,inputs=[python_code],outputs=[rust_code])
    run_python_button.click(run_python,inputs=[python_code],outputs=[python_result])
    run_rust_button.click(run_rust,inputs=[rust_code],outputs=[rust_result])


In [127]:
ui.launch(inbrowser=True)

### Customization

- Integrate other LLMs, both open-source or commercial, like Claude from Anthropic, for better performance or more tailored outputs.
- Customize the interface for smoother interactions and more flexibility in design.
- Allow for additional features like saving translations or adjusting settings to fit user preferences.
